In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
import numpy as np
import os

In [16]:
places_urls=[]
url = 'https://www.fis-ski.com/DB/cross-country/calendar-results.html?eventselection=&place=&sectorcode=CC&seasoncode=2020&categorycode=WC&disciplinecode=&gendercode=&racedate=&racecodex=&nationcode=&seasonmonth=X-2020&saveselection=-1&seasonselection='
response = requests.get(url, timeout=5)
content = BeautifulSoup(response.content, "html.parser")
tableHeader = content.find('div',id='calendardata')
rows = tableHeader.find_all('a', class_='pr-1 g-lg-1 g-md-1 g-sm-2 hidden-xs justify-left')
for row in rows:
    places_urls.append(str(row.get('href')))
    print (str(row.get('href')))

https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45161&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45164&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45165&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45166&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45167&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45170&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45168&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45169&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45171&seasoncode=2020
https://www.fis-ski.com/DB/general/event-details.html?sectorcode=CC&eventid=45173&seasoncode=2020
https://www.fis-ski.

Sätter season till rätt säsong utifrån url

In [17]:
tmp_url=places_urls[-1:]
season=tmp_url[0][-4:]

In [18]:
races_urls=[]
for url in places_urls:
    response = requests.get(url, timeout=5)
    content = BeautifulSoup(response.content, "html.parser")
    tableHeader = content.find('div',id='eventdetailscontent')
    rows = tableHeader.find_all('a', class_='px-1 g-lg-3 g-md-3 g-sm-4 g-xs-4 justify-left')
    for row in rows:
        races_urls.append(str(row.get('href')))
        print(str(row.get('href')))

https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34331
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34332
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34333
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34334
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34335
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34336
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34337
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34338
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=35980
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=35981
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34341
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34342
https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34343
https://www.fis-ski.com/D

In [19]:
Result=pd.DataFrame()
#races_urls = ['https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34331','https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34385','https://www.fis-ski.com/DB/general/results.html?sectorcode=CC&raceid=34381']
for url in races_urls:
    labels=[]
    response = requests.get(url, timeout=5)
    content = BeautifulSoup(response.content, "html.parser")
    event = content.find('div', class_='event-header__kind').getText()
    place = content.find('h1', class_ = re.compile('heading.*')).getText()
    date = content.find('span', class_='date__short').getText()
    event_id = url[-4:]
    result_type = content.find('div',class_ = 'event-header__subtitle').getText()
    print(event,place,date,result_type)
    tableHeader = content.find('div',{ "data-boundary" : "#events-info-results"})
    rows = tableHeader.find_all('div', class_ = re.compile('g-lg.*'))

    column_headers = [
    row.get_text() for row in rows
        ]

    data = content.find('div',id='events-info-results')
    data_rows = data.find_all('a', class_ = re.compile('table-row'))


    player_data_02 = []  # create an empty list to hold all the data

    for i in range(len(data_rows)):  # for each table row
            player_row = []  # create an empty list for each pick/player

            # for each table data element from each table row
            for row in data_rows[i].findAll('div', class_ = re.compile('g-lg.*')):
                # get the text content and append to the player_row
                player_row.append(row.getText())

            # then append each pick/player to the player_data matrix
            player_data_02.append(player_row)



    df = pd.DataFrame(player_data_02, columns=column_headers)
      
    df['Event'] = event
    df['Event ID'] = event_id
    df['Place'] = place
    df['Date'] = date
    df['Result Type'] = result_type
    df['Season'] = season

    frames = [Result, df]
    Result = pd.concat(frames,sort=False)
    
Result.info()


Women's SP  C Qual Ruka (FIN) Nov 29, 2019 Sprint Qualification
Men's SP  C Qual Ruka (FIN) Nov 29, 2019 Sprint Qualification
Women's SP  C Final Ruka (FIN) Nov 29, 2019 Stage World Cup
Men's SP  C Final Ruka (FIN) Nov 29, 2019 Stage World Cup
Women's 10 km C Ruka (FIN) Nov 30, 2019 Stage World Cup
Men's 15 km C Ruka (FIN) Nov 30, 2019 Stage World Cup
Women's 10 km F Pursuit Ruka (FIN) Dec 01, 2019 Stage World Cup
Men's 15 km F Pursuit Ruka (FIN) Dec 01, 2019 Stage World Cup
Women's Overall Standing Ruka (FIN) Dec 01, 2019 World Cup
Men's Overall Standing Ruka (FIN) Dec 01, 2019 World Cup
Women's Skiathlon 7.5/7.5km C/F Lillehammer (NOR) Dec 07, 2019 World Cup
Men's Skiathlon 15/15km C/F Lillehammer (NOR) Dec 07, 2019 World Cup
Women's Rel 4x5 km M Lillehammer (NOR) Dec 08, 2019 World Cup
Men's Rel 4x7.5 km M Lillehammer (NOR) Dec 08, 2019 World Cup
Women's SP  F Qual Davos (SUI) Dec 14, 2019 Sprint Qualification
Men's SP  F Qual Davos (SUI) Dec 14, 2019 Sprint Qualification
Men's SP  

In [20]:
Result

,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,Event ID,Place,Date,Result Type,Season,FIS points Time,(Rk.),Name,Tot. Time
0,1,31,3295157,\n SCARDONI Luc...,1991,\n\n\n\n\nITA\n\n,3:03.01,3:03.01,0.00,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,NaN,NaN,NaN,NaN
1,2,12,3535320,\n MAUBET BJORN...,1989,\n\n\n\n\nUSA\n\n,3:04.16,+1.15,7.54,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,NaN,NaN,NaN,NaN
2,3,23,3295193,\n LAURENT Gret...,1992,\n\n\n\n\nITA\n\n,3:05.15,+2.14,14.03,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,NaN,NaN,NaN,NaN
3,4,4,3535304,\n CALDWELL Sop...,1990,\n\n\n\n\nUSA\n\n,3:05.62,+2.61,17.11,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,NaN,NaN,NaN,NaN
4,5,27,3426112,\n STENSETH Ane...,1995,\n\n\n\n\nNOR\n\n,3:05.67,+2.66,17.44,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,39,3530772,\n LUSTGARTEN B...,1992,\n\n\n\n\nUSA\n\n,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020,NaN,NaN,NaN,NaN
47,48,52,3181007,\n VUORELA Mark...,1996,\n\n\n\n\nFIN\n\n,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020,NaN,NaN,NaN,NaN
48,49,56,3120063,\n SHANG Jincai...,1993,\n\n\n\n\nCHN\n\n,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020,NaN,NaN,NaN,NaN
49,50,38,3530711,\n MARTIN Adam ...,1994,\n\n\n\n\nUSA\n\n,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020,NaN,NaN,NaN,NaN


In [21]:
Result = Result.reset_index(drop=True)
Result['Nation']= Result['Nation'].str.replace('\n','')
Result['Athlete']= Result['Athlete'].str.replace('\n','')
Result['Athlete']= Result['Athlete'].str.strip()

columns=['Name','Tot. Time','(Rk.)','FIS points Time']
Result.drop(columns, inplace=True, axis=1)


Result

,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,Event ID,Place,Date,Result Type,Season
0,1,31,3295157,SCARDONI Lucia,1991,ITA,3:03.01,3:03.01,0.00,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
1,2,12,3535320,MAUBET BJORNSEN Sadie,1989,USA,3:04.16,+1.15,7.54,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
2,3,23,3295193,LAURENT Greta,1992,ITA,3:05.15,+2.14,14.03,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
3,4,4,3535304,CALDWELL Sophie,1990,USA,3:05.62,+2.61,17.11,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
4,5,27,3426112,STENSETH Ane Appelkvist,1995,NOR,3:05.67,+2.66,17.44,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6731,47,39,3530772,LUSTGARTEN Benjamin,1992,USA,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6732,48,52,3181007,VUORELA Markus,1996,FIN,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6733,49,56,3120063,SHANG Jincai,1993,CHN,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6734,50,38,3530711,MARTIN Adam,1994,USA,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020


In [24]:
if os.path.isfile('extract/raw-data.csv'):
    os.remove('extract/raw-data.csv')
Result.to_csv('extract/raw-data.csv',encoding='utf-8')
